In [ ]:
import networkx as nx
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
plt.axis("off")
import os

from analyze import *

#if true, will show generated plots in the notebook
#if false, will save to a folder called figs/synthetic-same (or -diff)
show = True 
if show:
    %matplotlib inline
exp = "diff" #"same" "diff"
dataset = "synthetic-%s" % exp
plot_labels = False

In [ ]:
#Graph: can't separate positive and negative components
G = nx.DiGraph()
if exp == "diff":
    G.add_nodes_from(range(7))
    G.add_edges_from([(0, 1, {'weight': 1}), (2, 3, {'weight': 1}), (2, 4, {'weight': 1})])#\, (5, 6, {'weight': 1})])
    G.add_edges_from([(0, 2, {'weight': -1})])#, (5, 7, {'weight': -1})])
    G.add_edges_from([(5, 6, {'weight': 1}), (5, 7, {'weight': -1})])
    focus_nodes = [0,5] #hand-engineered statistics or concatenated unsigned methods view as structurally equivalent, signed structural methods capture subtle difference
elif exp == "same":
    G.add_nodes_from(range(10))
    G.add_edges_from([(0, 1, {'weight': -1}), (1, 2, {'weight': 1}), (1, 3, {'weight': 1})])
    G.add_edges_from([(4, 5, {'weight': 1}), (5, 6, {'weight': -1}), (5, 7, {'weight': -1})])
    G.add_edges_from([(8, 9, {'weight': 1})])
    focus_nodes = [0,4,8,9,5]
else:
    G.add_nodes_from(range(11))
    G.add_edges_from([(1, 2, {'weight': 1}), (1, 3, {'weight': 1})])
    G.add_edges_from([(0, 1, {'weight': -1}), (4, 5, {'weight': -1}), (5, 6, {'weight': -1}), (5, 7, {'weight': -1})])
    #G.add_edges_from([(8, 0, {'weight': -1}), (4, 9, {'weight': 1}), (2, 10, {'weight': -1})])
    G.add_edges_from([(8, 0, {'weight': -1}), (10, 9, {'weight': -1}), (2, 10, {'weight': -1})])
    focus_nodes = [0,4] #hand-engineered statistics or concatenated unsigned methods view as structurally equivalent, signed structural methods capture subtle difference

In [ ]:
# #Draw the graph
COLOR_HIGHLIGHT = "#d7191c" #med red
COLOR_HIGHLIGHT2 = "#fdae61" #salmon
COLOR_HIGHLIGHT3 = "#f46d43" #orange
COLOR_REG = "#2c7bb6" #med blue
COLOR_REG2 = "#abd9e9" #light blue

node_colors = [COLOR_REG for u in G.nodes]
node_colors[focus_nodes[0]] = COLOR_HIGHLIGHT
node_colors[focus_nodes[1]] = COLOR_HIGHLIGHT2
if exp == "same":
    node_colors[focus_nodes[2]] = COLOR_REG2
    #node_colors[focus_nodes[3]] = COLOR_REG2
    node_colors[focus_nodes[4]] = COLOR_HIGHLIGHT3
if exp not in ["same", "diff"]:
    node_colors[8] = "blue"
    node_colors[10] = "purple"
    node_colors[2] = "yellow"

pos = nx.circular_layout(G)
# nodes
nx.draw_networkx_nodes(G,pos,
                       nodelist=G.nodes,
                       node_color=node_colors,
                       node_size=700)

#node labels
if plot_labels:
    node_ids = range(G.number_of_nodes())
    labels = dict(zip(node_ids, node_ids))
    nx.draw_networkx_labels(G,pos,labels,font_size=16)
#edges
pos_edges = []
neg_edges = []
for u,v in G.edges():
    if G[u][v]["weight"] >= 0: pos_edges.append( (u,v) )
    else: neg_edges.append( (u,v) )
print(pos_edges)
print(neg_edges)

#positive edges
nx.draw_networkx_edges(G,pos,
                       edgelist=pos_edges,
                       width=1.0,style="solid",arrows=True,arrowstyle='->',arrowsize = 40)#,edge_color='blue')
#negative edges
nx.draw_networkx_edges(G,pos,
                       edgelist=neg_edges,
                       style="dotted",
                       arrows=True,
                       arrowstyle='->',
                       arrowsize=40,
                       edge_color='purple')

#edge labels
edge_labels = {}
for u,v in G.edges:
    if G[u][v]["weight"] >= 0: label = "+"
    else: label = "-"
    edge_labels[(u,v)] = label
nx.draw_networkx_edge_labels(G,pos,edge_labels = edge_labels, font_size=12)#, clip_on = False)

if show:
    plt.show()
else:
    if not os.path.exists("figs/%s" % dataset): os.system("mkdir figs/%s" % dataset)
    plt.savefig("figs/%s/graph.png" % dataset)

In [ ]:
#Embed the graph using sNCE
embed_method = "snce" #
signed_xnetmf_emb = embed_data(nx.adjacency_matrix(G), method = embed_method, dim = 4)
viz_embed(signed_xnetmf_emb, method = embed_method, colors = node_colors, viz_list = focus_nodes, dataset = dataset, show = show, subsample = False)

In [ ]:
#Embed the graph using sRDE
embed_method = "srde"
signed_emb = embed_data(nx.adjacency_matrix(G), method = embed_method, dim = 4)
viz_embed(signed_emb, method = embed_method, colors = node_colors, viz_list = focus_nodes, dataset = dataset, show = show, subsample = False)

In [ ]:
#Degree features of graph
deg_feat = signed_network_features(nx.adjacency_matrix(G), method = "degrees")
viz_embed(deg_feat, method = "degrees", colors = node_colors, viz_list = focus_nodes, show = show, dataset = dataset, subsample = False)

In [ ]:
#Concatenate xNetMF for separate signs
embed_method = "sec-xnetmf"
concat_xnetmf_emb = signed_network_features(nx.adjacency_matrix(G), method = embed_method, dim = 8) #was 4
viz_embed(concat_xnetmf_emb, method = embed_method, colors = node_colors, viz_list = focus_nodes, show = show, dataset = dataset, subsample = False)